# Astelias - List
## Beachten Zukunft
* HCP & HCP fix gesetzt, da Sortierung falsch

In [2]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [3]:
# Read pdf into DataFrame
df = tabula.read_pdf("2017+(ENGLISH)+Disclosure+Code+Report_SWITZERLAND_12062018_V1.0.pdf", pages='all', lattice=True)

## Format Table

In [4]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty', 'name', 'location', 'country', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total']

#Remplate HCPs and HCOs
df_export['empty'] = df_export['empty'].str.replace("HCPs", "")
df_export['empty'] = df_export['empty'].str.replace("HCOs", "")
df_export['empty'] = df_export['empty'].fillna("")

#df_export[df_export['empty'] != '']

#Shift all where 'name' is empty
df_export[df_export['empty'] != ""] = df_export[df_export['empty'] != ""].shift(1, axis='columns')

#Remove rows which have no total
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Convert to Numbers
df_export['sponsorship'] = df_export['sponsorship'].str.replace("Na", "")
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Remove duplicate
df_export = df_export[df_export.duplicated("name", keep="first")]

#Remove empty
df_export.drop(columns='empty', inplace=True)

#Remove "name" in "adress" (like "Stiftung SONK","Stiftung SONK Rorschacher Strasse 150"
for index, row in df_export.iterrows():
    df_export.loc[index, 'address'] = row['address'].replace(row['name'] + ' ', '')
    
#Set Type
add_warning(manually=True)
df_export = add_type_by_index(df_export, 164)

#Turn Surname/Lastname
df_export.loc[df_export.type == 'hcp', 'name'] = revert_name(df_export[df_export.type == 'hcp']['name'])

#Remove Carination
df_export = remove_carination(df_export)

#Add missing columns
df_export = add_plz(df_export)
df_export = add_uci(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Export
export_list(df_export, 'astellas')

revert_name: Be sure: Only revert hcp, not hco!
Total nicht Summe der Werte
saved
